# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file = "../output_data/cities.csv"

cities_df = pd.read_csv(file)
cities_df.head()


,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Kapaa,22.08,-159.32,82.40,74,1,12.75,US,1602901251
1,Okhotsk,59.38,143.30,27.88,80,0,10.94,RU,1602901256
2,Hambantota,6.12,81.12,75.20,94,40,4.70,LK,1602901256
3,Arraial do Cabo,-22.97,-42.03,69.76,74,100,5.91,BR,1602901257
4,Qeshm,26.96,56.27,68.00,18,0,9.17,IR,1602901257


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# locations = cities_df[["Latitude", "Longitude"]]
# humidity = cities_df["Humidity"].astype(float)

# fig = gmaps.figure()

# # Create heat layer
# heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
#                                  dissipating=False, max_intensity=10,
#                                  point_radius=1)


# # Add layer
# fig.add_layer(heat_layer)

# # Display figure
# fig


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# use conditionals to narrow down weather data
ideal_weather_df = cities_df.loc[(cities_df['Max Temp'] < 85) & (cities_df['Max Temp'] > 65) & (cities_df['Humidity'] < 25) &(cities_df['Wind Speed'] < 10)] 

ideal_weather_df.to_csv("../output_data/ideal_weather.csv", index=False)




In [5]:
file2 = "../output_data/ideal_weather.csv"

ideal_weather = pd.read_csv(file2)
ideal_weather.head()

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Qeshm,26.96,56.27,68.00,18,0,9.17,IR,1602901257
1,Bilma,18.69,12.92,80.40,20,0,1.74,NE,1602901287
2,Saint George,37.10,-113.58,73.40,10,1,3.36,US,1602901121
3,Chabahar,25.29,60.64,75.52,24,0,8.16,IR,1602901299
4,Taoudenni,22.68,-3.98,83.55,21,0,8.41,ML,1602901301


In [6]:
ideal_weather["Hotel Name"] = ""

ideal_weather

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Qeshm,26.96,56.27,68.00,18,0,9.17,IR,1602901257,
1,Bilma,18.69,12.92,80.40,20,0,1.74,NE,1602901287,
2,Saint George,37.10,-113.58,73.40,10,1,3.36,US,1602901121,
3,Chabahar,25.29,60.64,75.52,24,0,8.16,IR,1602901299,
4,Taoudenni,22.68,-3.98,83.55,21,0,8.41,ML,1602901301,
5,San Jose,37.34,-121.89,82.99,18,40,5.82,US,1602901127,


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
target_coordinates = "26.96, 56.27"
target_radius = 50000
target_type = "lodging"
hotel = []

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)
#print(response)

places_data = response.json()

# Print the json (pretty printed)
#print(json.dumps(places_data, indent=4, sort_keys=True))
print(places_data["results"][0]["name"])
print(places_data["results"][0]["vicinity"])


Forough Bandar Hotel
Bandar Abbas, بلوار دانشگاه


In [13]:
hotel_df = []

params = {
    "radius": 50000,
    "types": "lodging",
    "key": g_key
}

for row in ideal_weather:
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    hotel_address = hotel_address.json()
    print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
#     try:
#         cities_pd.loc[index, "Airport Name"] = name_address["results"][0]["name"]
#         cities_pd.loc[index, "Airport Address"] = name_address["results"][0]["vicinity"]
#         cities_pd.loc[index, "Airport Rating"] = name_address["results"][0]["rating"]
#     except (KeyError, IndexError):
#         print("Missing field/result... skipping.")


TypeError: string indices must be integers

In [15]:
hotel = []

params = {
    "radius": 50000,
    "types": "lodging",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in ideal_weather.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        ideal_weather_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
